Chipping the CDL layer based on a geojson of area of interest. Produce a new chipped CDL tif file to use for reprojecting HLS

In [5]:
# this cell is clipping CDL CONUS layer to a smaller area captured in `cdl_aoi.geojson`

import fiona
import rasterio
import rasterio.mask

with fiona.open("/data/requirements/cdl_raw/cdl_aoi.geojson", "r") as shapefile:
    shapes = [feature["geometry"] for feature in shapefile]
    
with rasterio.open("/data/requirements/cdl_raw/2022_30m_cdls.tif") as src:
    out_image, out_transform = rasterio.mask.mask(src, shapes, crop=True)
    out_meta = src.meta
    colormap = src.colormap(1)
    
out_meta.update({"driver": "GTiff",
                 "height": out_image.shape[1],
                 "width": out_image.shape[2],
                 "transform": out_transform})

with rasterio.open("/data/requirements/2022_30m_cdls_clipped.tif", "w", **out_meta) as dest:
    dest.write(out_image)
    dest.write_colormap(1, colormap)